In [1]:
import os

from dotenv import load_dotenv
load_dotenv(r"H:\My_LangGraph_toturial\.env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["ACTIVELOOP_TOKEN"] = os.getenv("ACTIVELOOP_TOKEN")

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.9)
prompt = ChatPromptTemplate.from_template("What is a good name for a company that makes {product}?")

chain = prompt | model | StrOutputParser()

print(chain.invoke({"product": "eco-friendly water bottles"}))


GreenHydro EcoBottles


In [15]:
import uuid
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define a chat model
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": [AIMessage(content=response.content)]}

# Define the nodes we will cycle between
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Adding memory
memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

# Generate a unique thread ID for this conversation
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

# Function to process messages and print responses
def process_message(content):
    input_message = HumanMessage(content=content)
    for event in app.stream({"messages": [input_message]}, config, stream_mode="values"):
        event["messages"][-1].pretty_print()

# Start the conversation
print("AI: Tell me about yourself.")
process_message("Tell me about yourself.")

print("\nAI: What can you do?")
process_message("What can you do?")

print("\nAI: How can you help me with data analysis?")
process_message("How can you help me with data analysis?")

# Display the full conversation history
print("\nFull Conversation History:")
for message in app.checkpointer.get(thread_id)["messages"]:
    if isinstance(message, SystemMessage):
        print(f"System: {message.content}")
    elif isinstance(message, HumanMessage):
        print(f"Human: {message.content}")
    elif isinstance(message, AIMessage):
        print(f"AI: {message.content}")


AI: Tell me about yourself.
================================ Human Message =================================

Tell me about yourself.
================================== Ai Message ==================================

I am a language model AI created by OpenAI. I am designed to assist with a wide range of tasks, including answering questions, providing information, and engaging in conversation. I am constantly learning and improving my abilities to better assist users like you. How can I help you today?

AI: What can you do?
================================ Human Message =================================

What can you do?
================================== Ai Message ==================================

I can help with a variety of tasks, such as answering questions, providing information on a wide range of topics, assisting with writing and editing text, generating ideas or suggestions, and engaging in conversation. I can also provide recommendations, offer guidance, and assist with prob

TypeError: 'UUID' object is not subscriptable

In [18]:
checkpoint_tuple = app.checkpointer.get_tuple(thread_id=str(thread_id))
messages = checkpoint_tuple.state["messages"]
message

TypeError: InMemorySaver.get_tuple() got an unexpected keyword argument 'thread_id'